In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from glob import glob
from skimage.feature import hog
from skimage import exposure
from skimage import segmentation
from sklearn.cluster import KMeans
from sklearn.model_selection import ShuffleSplit
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn import naive_bayes
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn import metrics
from funcionesProyecto import funcionLeerClase, funcionLeerTodasClases, ecualizacionAdaptativa, imagen_media_color, quitarFondo, generarHogClase, generarCsv

<br/><br/>**<u>PREPARAMOS LOS DATOS</u>:**<br/><br/><br/><br/>

In [2]:
datosLista = funcionLeerTodasClases('train')

En train/10c, hay 254 ejemplos
En train/1c, hay 77 ejemplos
En train/1e, hay 248 ejemplos
En train/20c, hay 227 ejemplos
En train/2c, hay 25 ejemplos
En train/2e, hay 45 ejemplos
En train/50c, hay 131 ejemplos
En train/5c, hay 284 ejemplos


In [13]:
fd = hog(datosLista[0][0], orientations=8, pixels_per_cell=(8, 8),cells_per_block=(7, 7))
numeroColumnas = fd.size
datos = np.empty((0,numeroColumnas+1))
for i,clase in enumerate(datosLista):
    array = generarHogClase(clase)
    array = np.hstack((array, np.full((array.shape[0], 1), i)))
    print('La clase {} tiene {} ejemplos'.format(i,array.shape[0]))
    datos = np.vstack((datos, array))

La clase 0 tiene 254 ejemplos
La clase 1 tiene 77 ejemplos
La clase 2 tiene 248 ejemplos
La clase 3 tiene 227 ejemplos
La clase 4 tiene 25 ejemplos
La clase 5 tiene 45 ejemplos
La clase 6 tiene 131 ejemplos
La clase 7 tiene 284 ejemplos


In [10]:
X = datos[:,:-1]
print('En total hay {} ejemplos, con {} caracteristicas'.format(X.shape[0], X.shape[1]))
y = datos[:,-1].reshape(-1,1)

En total hay 1291 ejemplos, con 56448 caracteristicas


In [11]:
datosTest = funcionLeerClase('public_test')
print('Hay {} ejemplos de test'.format(len(datosTest)))

Hay 964 ejemplos de test


In [12]:
test = generarHogClase(datosTest)

<br/><br/>**<u>PROBAMOS SVM</u>:**<br/><br/><br/><br/>

In [ ]:
#Entrenamos con todos los ejemplos de train
C_values = [0.1,1,2,3,4,5,6,7]
gamma_values = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
params = {'C': C_values, 'gamma': gamma_values, 'kernel': kernel}
rs = ShuffleSplit(n_splits=1, test_size= 0.2, random_state=0)
svc = svm.SVC()
clf = GridSearchCV(svc, params, cv=rs, n_jobs=-1)
clf = clf.fit(X,y.ravel())
C_best = clf.best_params_['C']
gamma_best = clf.best_params_['gamma']
kernel_best = clf.best_params_['kernel']
print("Mejores parámetros C: {}, gamma: {}, kernel: {}".format(C_best, gamma_best, kerne_best))
svc = svm.SVC(gamma=gamma_best, C=C_best, kernel=kernel_best)
svc = svc.fit(X,y.ravel())
accTrain = svc.score(X,y)
print(accTrain)

In [ ]:
test_labels = svc.predict(test)

<br/><br/>**<u>GENERAMOS LOS RESULTADOS</u>:**<br/><br/><br/><br/>

In [ ]:
generarCsv('public_test', test_labels)